In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [2]:

# open text file and read in data as `text`
with open('../input/anna.txt', 'r') as f:
    text = f.read()

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [4]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

In [5]:
encoded[:100]

array([64, 32, 74, 52, 59,  7, 50, 27, 47, 67, 67, 67, 36, 74, 52, 52, 17,
       27, 57, 74, 35, 24, 29, 24,  7, 53, 27, 74, 50,  7, 27, 74, 29, 29,
       27, 74, 29, 24, 23,  7, 81, 27,  7, 82,  7, 50, 17, 27, 43, 16, 32,
       74, 52, 52, 17, 27, 57, 74, 35, 24, 29, 17, 27, 24, 53, 27, 43, 16,
       32, 74, 52, 52, 17, 27, 24, 16, 27, 24, 59, 53, 27,  4, 51, 16, 67,
       51, 74, 17,  8, 67, 67, 76, 82,  7, 50, 17, 59, 32, 24, 16])

In [6]:

def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [7]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [8]:

def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [9]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [10]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[64 32 74 52 59  7 50 27 47 67]
 [53  4 16 27 59 32 74 59 27 74]
 [ 7 16  6 27  4 50 27 74 27 57]
 [53 27 59 32  7 27 63 32 24  7]
 [27 53 74 51 27 32  7 50 27 59]
 [63 43 53 53 24  4 16 27 74 16]
 [27 31 16 16 74 27 32 74  6 27]
 [77 20 29  4 16 53 23 17  8 27]]

y
 [[32 74 52 59  7 50 27 47 67 67]
 [ 4 16 27 59 32 74 59 27 74 59]
 [16  6 27  4 50 27 74 27 57  4]
 [27 59 32  7 27 63 32 24  7 57]
 [53 74 51 27 32  7 50 27 59  7]
 [43 53 53 24  4 16 27 74 16  6]
 [31 16 16 74 27 32 74  6 27 53]
 [20 29  4 16 53 23 17  8 27 54]]


In [11]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [12]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [13]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [14]:
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [15]:
batch_size = 128
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/20... Step: 10... Loss: 3.2431... Val Loss: 3.1641
Epoch: 1/20... Step: 20... Loss: 3.1431... Val Loss: 3.1263
Epoch: 1/20... Step: 30... Loss: 3.1391... Val Loss: 3.1203
Epoch: 1/20... Step: 40... Loss: 3.1129... Val Loss: 3.1187
Epoch: 1/20... Step: 50... Loss: 3.1401... Val Loss: 3.1173
Epoch: 1/20... Step: 60... Loss: 3.1141... Val Loss: 3.1142
Epoch: 1/20... Step: 70... Loss: 3.1026... Val Loss: 3.1108
Epoch: 1/20... Step: 80... Loss: 3.1132... Val Loss: 3.1023
Epoch: 1/20... Step: 90... Loss: 3.1000... Val Loss: 3.0836
Epoch: 1/20... Step: 100... Loss: 3.0562... Val Loss: 3.0389
Epoch: 1/20... Step: 110... Loss: 3.0057... Val Loss: 2.9890
Epoch: 1/20... Step: 120... Loss: 2.9047... Val Loss: 2.9017
Epoch: 1/20... Step: 130... Loss: 2.8377... Val Loss: 2.8277
Epoch: 2/20... Step: 140... Loss: 2.7464... Val Loss: 2.7057
Epoch: 2/20... Step: 150... Loss: 2.6430... Val Loss: 2.5964
Epoch: 2/20... Step: 160... Loss: 2.5721... Val Loss: 2.5421
Epoch: 2/20... Step: 170... Loss:

Epoch: 10/20... Step: 1350... Loss: 1.4015... Val Loss: 1.4287
Epoch: 10/20... Step: 1360... Loss: 1.4073... Val Loss: 1.4298
Epoch: 10/20... Step: 1370... Loss: 1.3992... Val Loss: 1.4262
Epoch: 10/20... Step: 1380... Loss: 1.4394... Val Loss: 1.4240
Epoch: 10/20... Step: 1390... Loss: 1.4459... Val Loss: 1.4224
Epoch: 11/20... Step: 1400... Loss: 1.4359... Val Loss: 1.4222
Epoch: 11/20... Step: 1410... Loss: 1.4555... Val Loss: 1.4165
Epoch: 11/20... Step: 1420... Loss: 1.4417... Val Loss: 1.4119
Epoch: 11/20... Step: 1430... Loss: 1.4078... Val Loss: 1.4144
Epoch: 11/20... Step: 1440... Loss: 1.4327... Val Loss: 1.4127
Epoch: 11/20... Step: 1450... Loss: 1.3608... Val Loss: 1.4104
Epoch: 11/20... Step: 1460... Loss: 1.3874... Val Loss: 1.4085
Epoch: 11/20... Step: 1470... Loss: 1.3818... Val Loss: 1.4079
Epoch: 11/20... Step: 1480... Loss: 1.4046... Val Loss: 1.4049
Epoch: 11/20... Step: 1490... Loss: 1.3911... Val Loss: 1.4044
Epoch: 11/20... Step: 1500... Loss: 1.3805... Val Loss:

Epoch: 20/20... Step: 2660... Loss: 1.2505... Val Loss: 1.3023
Epoch: 20/20... Step: 2670... Loss: 1.2581... Val Loss: 1.2998
Epoch: 20/20... Step: 2680... Loss: 1.2563... Val Loss: 1.2984
Epoch: 20/20... Step: 2690... Loss: 1.2390... Val Loss: 1.3000
Epoch: 20/20... Step: 2700... Loss: 1.2495... Val Loss: 1.2977
Epoch: 20/20... Step: 2710... Loss: 1.2172... Val Loss: 1.2984
Epoch: 20/20... Step: 2720... Loss: 1.2210... Val Loss: 1.2991
Epoch: 20/20... Step: 2730... Loss: 1.2158... Val Loss: 1.2961
Epoch: 20/20... Step: 2740... Loss: 1.2103... Val Loss: 1.2977
Epoch: 20/20... Step: 2750... Loss: 1.2250... Val Loss: 1.2984
Epoch: 20/20... Step: 2760... Loss: 1.2131... Val Loss: 1.2971
Epoch: 20/20... Step: 2770... Loss: 1.2529... Val Loss: 1.2950
Epoch: 20/20... Step: 2780... Loss: 1.2798... Val Loss: 1.2911


In [16]:
# change the name, for saving multiple files
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [17]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [18]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [19]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna's soughs is so thanked in him of
that man of the pain. I don't know those that were in that
sister and said in society, and they see you say that, it's only a port and trunts to be as though I were the stream and things
is no one always
asking it on the first, and the cans to make, but she did not know what I was strange, but I should have to speak to her. There were the player to be some at it in
her freedom, and as he has been an effact and something face to the subjects. And it's all so to blame from
the
state, and that is intallability, all
well," said Levin,
as she was not self-in spite of the dissitition to him, with a time at a serform. The charm had been felt a servants of her hands with
an
imaginatien. The meaning he could not, holding her. He felt that she had been so that this hand he was simply to be asked to the capparal
from his hand on the
chain,
were sat for him. The soul of
this second things were too as she
was not the
side of the servants, and something of all t

In [20]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_20_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

In [21]:
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said.

"I had now to be a servant. How am I all as
soon as you was a straight of my friends or three circoms. Would you do with you?"

"Well, and I can't believe it's to make it in the second as to take anyone, and so. What would you say what is the more the sense on the process of
a perplexity, thrusiness to me to think it impossible, and I am good--as you says, and tell me and
think of me.
And. I want
the servor, to speak. This cass they has not
been to greeting the
men, as you are counting
to the childishs to the counting--what's with the steal," he said, "but what was to take intellect in the foonmation?" she added, and with a clear of the conversation of his wealiness and still
standing at his head and
hearing it,
and so would not live to broke his heart. Telling the feeling of
such a long ago, she
could not be too through their hands, that her state, to the strends of her hands. Alexey Alexandrovitch was desired to the sort, a driver in-the starces, thir traceful serenc